In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

# 1. Import Data
### import original dataset

In [ ]:
print (os.path.abspath('.'))
os.chdir("C:\\Users\\D.Sia\\Desktop\\Redshift\\Decision Tree")
print (os.getcwd())

In [ ]:
data = pd.read_csv('match_COSMOS_99_with_err.csv')
#print(data.columns.values)

### 1.1 column names (find magnitude)

In [ ]:
print(data)

In [ ]:
print(data)
columns_name = data.columns.values
#print(columns_name)
mag_name_list=[]
for c in columns_name:
    if "_MAG_" in c:
        mag_name_list.append(c)
print(mag_name_list)

### 1.2 data without zspec<=0

In [ ]:
cosmos_data = data[data["zspec"]>0]
#print(len(cosmos_data))

### 1.3 zspec data plot (to verify the previous step)

In [ ]:
plt.figure()
plt.scatter(cosmos_data["zspec"],cosmos_data["zspec"])

### 1.4 decision_tree_data(find IB and the IB value)


#### —— IB: IA767_MAG_APER2 
#### —— IB_value = 22.6 

In [ ]:
#print(cosmos_data["IA767_MAG_APER2"])
for i in np.arange(21,25,0.1):
    print("Range:(",round(i,2),",",round(i+0.1,2),") Counts:",len(cosmos_data[(cosmos_data.IA767_MAG_APER2> i) & (cosmos_data.IA767_MAG_APER2 < i+0.1)]))
#print(len(cosmos_data[(cosmos_data.IA767_MAG_APER2>22.4) & (cosmos_data.IA767_MAG_APER2 < 22.5)]))


In [ ]:
IB = "IA767_MAG_APER2"
IB_value = 22.6
decision_tree_data = cosmos_data[cosmos_data[IB] < IB_value]
print(decision_tree_data)

# 2. Decision Tree

### 2.1 input dataset

In [ ]:
N = len(decision_tree_data)
X = np.zeros((N, len(mag_name_list)-1))
for i in range(len(mag_name_list)-1):
    X[:, i] = decision_tree_data[mag_name_list[i]] - decision_tree_data[mag_name_list[i+1]]
z = decision_tree_data['zspec']

### 2.2 training and test set (8:2)

In [ ]:
Ntrain = int(8 * N / 10)
Xtrain = X[:Ntrain]
ztrain = z[:Ntrain]
Xtest = X[Ntrain:]
ztest = z[Ntrain:]

### 2.3  training and test set description

In [ ]:
print ("training sample size: ",len(Xtrain))
print ("test sample size    : ",len(Xtest))
print ("train   +   test    = ", N )
print ("train / sample =", round(len(Xtrain)/(len(Xtrain)+len(Xtest)),1))
print ("test  / sample =", round(len(Xtest)/(len(Xtrain)+len(Xtest)),1))

### 2.3.1 decision Tree

In [ ]:
from sklearn import tree
clf = tree.DecisionTreeRegressor()
clf = clf.fit(Xtrain, ztrain)
# prediction on the test sample
zpred = clf.predict(Xtest)

In [ ]:
# distribution of redshifts predicted from the training sample
plt.figure(figsize=(8,6))
plt.hist(zpred,bins=30,range=(0.,1.5))
plt.xlabel(r'$z_{\rm{predicted}}$',fontsize=20)
plt.ylabel('counts',fontsize=20)
plt.show()

# distribution of real spectroscopic redshifts 
# on the test sample
plt.figure(figsize=(8,6))
plt.hist(ztest,bins=30,range=(0.,1.5))
plt.xlabel(r'$z_{\rm{spec}}$_test',fontsize=20)
plt.ylabel('counts',fontsize=20)
plt.show()

In [ ]:
plt.figure(figsize=(8,6))
plt.scatter(zpred,ztest,marker=".",s=6,color="steelblue")
plt.plot([0.,2.],[0.,2.],color="darkslategray")
plt.xlim(0.,1.2)
plt.ylim(0.,1.2)
plt.xlabel(r'$z_{\rm{spec}}$',fontsize=20)
plt.ylabel(r'$z_{\rm{predicted}}$',fontsize=20)
plt.show()

### 2.3.2 Predict the entire sample and compare it with an estimate of the photometric redshift

In [ ]:
N_all = len(data)
X_all = np.zeros((N_all, len(mag_name_list)-1))
for i in range(len(mag_name_list)-1):
    X_all[:, i] = data[mag_name_list[i]] - data[mag_name_list[i+1]]
z_all = data['zspec']
z_predict_all = clf.predict(X_all)

In [ ]:
# distribution of redshifts predicted for the entire sample
plt.figure(figsize=(8,6))
plt.hist(z_predict_all,bins=40,range=(0.,4))
plt.xlabel('z predicted (all sample)',fontsize=20)
plt.ylabel('counts',fontsize=20)
plt.show()

# to test the entire sample we can't use z_spec simply because we don't have measurements
# but we have an estimate of the photometric redshift from the COSMOS sample.
plt.figure(figsize=(8,6))
plt.hist(decision_tree_data['Z_MINCHI2'],bins=30,range=(0.,4))
plt.xlabel(r'$z_{\rm{photo}}$ (all sample)',fontsize=20)
plt.ylabel('counts',fontsize=20)
plt.show()

In [ ]:
plt.figure(figsize=(8,6))
plt.scatter(data['Z_MINCHI2'],z_predict_all,marker=".",s=1)
plt.plot([0.,2.],[0.,2.])
plt.xlim(0.,1.5)
plt.ylim(0.,1.5)
plt.xlabel(r'$z_{\rm{photo}}$',fontsize=20)
plt.ylabel(r'$z_{\rm{predicted}}$',fontsize=20)
plt.title('All sample',fontsize=20)
plt.show()